AdaBoost - это один из алгоритмов бустинга. Это ансамбль, в котором обучается много копий более слабой модели ("weak learner"). На каждом следующем шаге новый weak learner больше фокусируется на тех примерах *из обучающего датасета*, которые не смогли предсказать предыдущие weak learner'ы. Для каждого weak learner'а рассчитывается вес. Финальное предсказание всего ансамбля - это взвешенное среднее предсказаний всех weak learner'ов (или взвешенная медиана для AdaBoost.R2).

Отсюда следуют сразу две вещи. Во-первых если weak learner смог выучить все обучающие примеры (например, это решающее дерево неограниченной глубины), то бустинг работать не будет: следующему алгоритму будет не на чем фокусироваться (технически произойдет деление на ноль). Во-вторых если weak learner - это линейная модель, то бустинг не будет иметь смысла: взвешенное среднее линейных моделей - это также линейная модель. Но в качестве weak learner'а вполне подойдет решающее дерево небольшой глубины.

## Варианты алгоритма AdaBoost

AdaBoost был предложен в [статье]($A Decision-Theoretic Generalization of On-Line Learning and an Application to Boosting$), где описаны 4 его варианта. Позже были предложены и другие варианты алгоритма. Sklearn [реализует](https://scikit-learn.org/stable/modules/ensemble.html#adaboost) версии AdaBoost-SAMME.R для классификации и AdaBoost.R2 для регрессии. Итак, 3 варианта алгоритма лежат перед нами как 3 завернутых конфеты, давайте их разворачивать.

## AdaBoost для бинарной классификации

Из статьи "[A Decision-Theoretic Generalization of On-Line Learning and an Application to Boosting]($A Decision-Theoretic Generalization of On-Line Learning and an Application to Boosting$)".

**Инициализация алгоритма**

Имеем m обучающих примеров, целевые данные сбалансированы и принадлежат множеству {-1, +1}.
Имеем некий обучаемый алгоритм (weak learner) для решения задачи бинарной классификации.
Каждому примеру ставим в соответствие вес, инициализируем его значением 1/m.
Выбираем количество шагов T.

**For t = 1, ..., T:**

1. Обучаем weak learner на обучающих примерах, используя веса. Например, в sklearn это соответствует параметру class_weights в методе .fit() для многих моделей. Если процесс обучения не поддерживает такой параметр, то используем для обучения случайное подмножество обучающего датасета, используя веса как вероятности выбора.

2. Получаем предсказания weak learner'а на *обучающем датасете* и оцениваем взвешенную ошибку $\epsilon_t$, используя веса. Формула: `((y_true != y_pred)*weights).sum()`. Предполагаем, что $\epsilon_t$ меньше 0.5 (в противном случае алгоритм хуже случайного угадывания, и мы можем просто инвертировать его предсказания).

3. Сохраняем текущий weak learner как $h_t(x)$ и ставим ему в соответствие вес $\alpha_t = \frac{1}{2}\ln(\frac{1-\epsilon_t}{\epsilon_t})$. Эта функция от $\epsilon_t$ равна 0 при $\epsilon_t = 0.5$ и стремится к бесконечности при $\epsilon_t \rightarrow 0$. То есть чем выше взвешенная точность алгоритма, тем больше $\alpha_t$. Заметим, что если алгоритм угадал все примеры, то получаем бесконечный вес и не можем продолжать бустинг. Если же алгоритм имеет точность 0.5 (точность случайного угадывания), то его вес будет равен 0.

4. Вес тех примеров, которые неверно предсказаны текущим weak learner'ом, умножаем на $\exp(\alpha_t)$. Веса верно угаданных примеров делим на это же число. После этого нормализуем веса, чтобы их сумма равнялась единице (делим каждый вес на сумму всех весов).

**Инференс алгоритма:**

Считаем взвешенную сумму предсказаний всех weak learner'ов, используя их нормализованные веса (чтобы сумма весов равнялась 1). Затем применяем операцию sign, чтобы получить ответ -1 или +1. При этом все weak learner'ы выдают ответы -1 или +1 (а не действительное число между этими значениями).

Заметим, что если мы не будем обновлять веса примеров, то AdaBoost превратится в бэггинг (точнее, в один из его вариантов): каждый weak learner будет обучаться на случайной подвыборке с возвращением.

## AdaBoost-SAMME для мультиклассовой классификации

Из статьи "[Multi-class AdaBoost]($Multi-class AdaBoost$)", реализован в `sklearn.ensemble.AdaBoostClassifier`. Этот алгоритм работает в целом аналогично описанному выше, но отличается одной деталью.

Мы точно так же считаем взвешенную ошибку $\epsilon_t$, используя формулу `((y_true != y_pred)*weights).sum()`. Сохраняя текущий weak learner, мы ставим ему в соответствие вес $\alpha_t = \ln(\frac{1-\epsilon_t}{\epsilon_t}) + \ln(K - 1)$. Здесь $K$ - количество классов. Дополнительное слагаемое добавлено с той целью, чтобы $\alpha_t$ было больше нуля тогда и только тогда, когда точность weak learner'а выше точности случайного угадывания. Если ошибка стремится к 0, то $\alpha_t$ стремится к бесконечности.

**Инференс алгоритма:**

Считаем взвешенную сумму предсказаний всех weak learner'ов (в виде вероятностей классов), используя их веса. Далее выполняется операция argmax для нахождения класса с наибольшей вероятностью (поэтому нормализация весов weak learner'ов здесь не требуется - argmax нечувствителен к масштабированию).

**AdaBoost-SAMME.R**

Авторами также предложен вариант алгоритма AdaBoost-SAMME.R ("Real"). Отличие алгоритма SAMME.R в том, что при расчете $\epsilon_t$ и $\alpha_t$ используются не финальные предсказания weak learner'а (в виде номера класса), а вероятности классов. Скажем, если weak learner'а выдал вероятности классов [0.55, 0.45, 0] (первый случай) или [1 0 0] (второй случай), то для SAMME эти случаи будут эквивалентны, но не для SAMME.R. Интуитивно понятно, что в первом случае weak learner работает на данном примере хуже, чем во втором. Поэтому SAMME.R, учитывающий это, сходится быстрее SAMME и применяется по умолчанию в `sklearn.ensemble.AdaBoostClassifier`.

## AdaBoost.R2 для регрессии

Из статьи "[Improving Regressors using Boosting Techniques]($Improving Regressors Using Boosting Techniques$)", реализован в `sklearn.ensemble.AdaBoostRegressor`.

**Инициализация алгоритма**

Имеем m обучающих примеров, целевые данные принадлежат множеству действительных чисел.
Имеем некий обучаемый алгоритм (weak learner) для решения задачи регрессии.
Каждому примеру ставим в соответствие вес $w_i$, инициализируем его значением 1. Нормализованные веса (то есть каждый вес, деленый на сумму всех весов) будем обозначать $p_i$.

**For t = 1, ..., inf (пока loss L не станет меньше 0.5):**

1. Обучаем weak learner на случайной подвыборке с возвращением из обучающего датасета, используя веса как вероятности выбора. Размер выборки равен размеру обучающего датасета.

2. Получаем предсказания weak learner'а на обучающем датасете и считаем функцию потерь для каждого примера. Функция потерь может быть любой, но ее значения должны находиться в диапазоне [0, 1]. Например, автор предлагает такой вариант:

$L_i = \frac{(y_{true} - y_{pred})^{2}}{\max_{i} (y_{true} - y_{pred})^{2}}$

Интересно, что согласно такой формуле, если мы возьмем все пары ($y_{true}$, $y_{pred}$) и в каждой паре изменим $y_{pred}$ так, чтобы отклонение от верного ответа увеличилось в 2 раза, то $L_i$ не изменится.

3. Считаем суммарную взвешенную функцию потерь:

$L = \sum L_i p_i$

4. Считаем значение $\beta_t = \frac{L}{1-L}$. Согласно автору, $\beta_t$ соответствует степени уверенности в предсказаниях: низкое $\beta_t$ означает высокую уверенность. Хотя не совсем понятно что понимать под "уверенностью".

Сохраняем текущий weak learner как $h_t(x)$ и ставим ему в соответствие вес $\alpha_t = \ln(1 / \beta_t)$.

5. Обновляем веса каждого обучающего примера по формуле:

$w_i \leftarrow w_i*\beta_t^{1-L_i}$

**Инференс алгоритма:**

Мы имеем набор weak learner'ов и их весов. Получив предсказания weak learner'ов, мы используем *взвешенную медиану* для получения финального ответа. Для этого сначала мы нормализуем веса weak learner'ов (чтобы сумма равнялась 1) и сортируем все предсказания weak learner'ов в порядке возрастания. Таким образом мы получаем цепочку предказаний, в которой каждое следующее предсказание больше либо равно предыдущему, и каждому предсказанию поставлен в соответствие вес. Мы проходим по этой цепочке от наименьшего к наибольшему, накапливая суммарный вес пройденных предсказаний до тех пор, пока суммарный вес не станет больше 0.5. Последнее из пройденных предсказаний считаем финальным ответом.

Такой интересный алгоритм взвешенного усреднения предсказаний алгоритмов регрессии можно использовать и за пределами AdaBoost. Его преимущество в том, что медиана устойчива к выбросам: даже если один из алгоритмов предскажет миллиард, то на ответ это не повлияет, а при простом взвешенном усреднении повлияло бы.